# Building the dataset for the 2024 mexican election

The purpose of this file is to translate the raw data, as included in the New_DB file, into a goup of datasets that resemble the sample photo included in the project. This sample is originated by the collection of the number of posts, and reactions to each one as for different social media profiles for each candidate, Xóchitl Gálvez, Claudia Sheinbaum, and Álvarez Mainez in this upcoming 2024 election.

The New_DB file is updated on a daily basis to collect the presence of the candidates in different social media platforms, and the reponse from their audience to it. Update dates are intended to be part as the commit message.

In [1]:
# imports
import os
import pandas as pd
import openpyxl

In [3]:
# We will treat Claudia as candidate 1, Galvez as 2, and Mainez as 3

In [10]:
# The days prior to a date for which we´ll sum the number of reactions
days = [1,2,3,4,5,6,7,14,21,28]

# Dates of published polls to read (as reported by Oraculus.mx)
date_strings = [
    "2023-09-15", "2023-09-15", "2023-10-15", "2023-10-15", "2023-10-15", "2023-10-15", "2023-10-15", "2023-10-15", "2023-10-15", "2023-11-15", "2023-11-15", "2023-11-15", "2023-11-15", "2023-11-15", "2023-11-15", "2023-11-15", "2023-11-15", "2023-11-15", "2023-12-15", "2023-12-15", "2023-12-15", "2023-12-15", "2023-12-15", "2023-12-15", "2023-12-15", "2023-12-15", "2023-12-15", "2023-12-15", "2024-01-15", "2024-01-15", "2024-01-15", "2024-01-15", "2024-01-15", "2024-01-15", "2024-01-15"
]

# Target variables for each polling release (reported share of preference)
targets_claudia = [49, 53, 59, 46, 50, 55, 48, 60, 57, 48, 57, 66, 50, 49, 54, 50, 52, 46, 51, 55, 65, 52, 52, 54, 50, 53, 57, 46, 54, 66, 48, 55, 61, 51, 63]

targets_galvez = [26, 34, 21, 28, 27, 24, 27, 17, 22, 24, 22, 14, 31, 23, 24, 27, 25, 25, 27, 22, 13, 30, 30, 23, 25, 26, 23, 24, 27, 14, 32, 27, 24, 27, 22]

targets_maynez = [18, 7, 5, 8, 15, 9, 10, 9, 7, 8, 7, 6, 7, 17, 8, 10, 10, 14, 3, 5, 6, 7, 10, 9, 6, 5, 9, 11, 3, 2, 10, 2, 5, 5, 4]

# Polls correspoding to a date
polls = ["Enkoll", "GEA-ISA", "Covarrubias", "El FInanciero", "Enkoll", "Mendoza Blanco", "Mitofsky", "Parametría", "Simo", "Buendía y Márquez", "Covarrubias", "Demotecnia", "El Financiero", "Enkoll", "Mendoza Blanco", "Mitofsky", "Parametría", "Reforma", "Berumen", "Covarrubias", "Demotecnia", "El Financiero", "GEA-ISA", "Mendoza Blanco", "Mitofsky", "Parametría", "Simo", "Citibanamex", "Berumen", "Demotecnia", "El Financiero", "Enkoll", "Mendoza Blanco", "Mitofsky", "Simo"]

#Setting columns to use (see New_DB)
columns = ['XPosts', 'Xcomments', 'XRts', 'Xlikes', 'XCommsPPost', 'XRTsPPost', 'XlikesPPost', 'FBPosts', 'FBReactions', 'FBComments', 'FBShares', 'FBReactsPPost', 'FBCommsPPost', 'FBSharesPPost', 'IGPosts', 'IGLikes', 'IGLikesPPost', 'YTPosts', 'YTViews', 'YTViewsPPost']

In [11]:
# Does every day have a poll? Does every poll have a percentage?
len(date_strings), len(targets_claudia), len(targets_galvez), len(polls), len(targets_maynez)

(35, 35, 35, 35, 35)

In [12]:
# Create a set of tuples associating dates with corresponding polls
date_poll_set_claudia = set(zip(date_strings, polls, targets_claudia))
for i, n, j in date_poll_set_claudia:
  print(f"Date: {i}, Poll: {n}, Result: {j}")

print(len(date_poll_set_claudia))

Date: 2023-12-15, Poll: Mendoza Blanco, Result: 54
Date: 2023-10-15, Poll: Covarrubias, Result: 59
Date: 2024-01-15, Poll: El Financiero, Result: 48
Date: 2023-11-15, Poll: Covarrubias, Result: 57
Date: 2024-01-15, Poll: Berumen, Result: 54
Date: 2023-10-15, Poll: Mitofsky, Result: 48
Date: 2023-12-15, Poll: El Financiero, Result: 52
Date: 2023-11-15, Poll: Parametría, Result: 52
Date: 2024-01-15, Poll: Mendoza Blanco, Result: 61
Date: 2023-11-15, Poll: Reforma, Result: 46
Date: 2023-11-15, Poll: El Financiero, Result: 50
Date: 2023-11-15, Poll: Demotecnia, Result: 66
Date: 2023-10-15, Poll: Parametría, Result: 60
Date: 2023-11-15, Poll: Mendoza Blanco, Result: 54
Date: 2023-09-15, Poll: Enkoll, Result: 49
Date: 2023-12-15, Poll: Demotecnia, Result: 65
Date: 2023-12-15, Poll: Simo, Result: 57
Date: 2023-11-15, Poll: Enkoll, Result: 49
Date: 2023-12-15, Poll: Mitofsky, Result: 50
Date: 2024-01-15, Poll: Simo, Result: 63
Date: 2023-10-15, Poll: Simo, Result: 57
Date: 2023-11-15, Poll: Bu

In [13]:
# Create a set of tuples associating dates with corresponding polls
date_poll_set_galvez = set(zip(date_strings, polls, targets_galvez))
for i, n, j in date_poll_set_galvez:
  print(f"Date: {i}, Poll: {n}, Result: {j}")
print(len(date_poll_set_galvez))

Date: 2023-11-15, Poll: Parametría, Result: 25
Date: 2023-11-15, Poll: Mendoza Blanco, Result: 24
Date: 2023-11-15, Poll: Reforma, Result: 25
Date: 2023-12-15, Poll: Mendoza Blanco, Result: 23
Date: 2023-12-15, Poll: Citibanamex, Result: 24
Date: 2023-11-15, Poll: Mitofsky, Result: 27
Date: 2023-11-15, Poll: Buendía y Márquez, Result: 24
Date: 2023-11-15, Poll: Demotecnia, Result: 14
Date: 2024-01-15, Poll: El Financiero, Result: 32
Date: 2023-10-15, Poll: Enkoll, Result: 27
Date: 2024-01-15, Poll: Mendoza Blanco, Result: 24
Date: 2023-10-15, Poll: El FInanciero, Result: 28
Date: 2024-01-15, Poll: Simo, Result: 22
Date: 2023-12-15, Poll: Demotecnia, Result: 13
Date: 2023-12-15, Poll: Covarrubias, Result: 22
Date: 2023-12-15, Poll: Berumen, Result: 27
Date: 2023-12-15, Poll: El Financiero, Result: 30
Date: 2023-10-15, Poll: Parametría, Result: 17
Date: 2024-01-15, Poll: Mitofsky, Result: 27
Date: 2023-11-15, Poll: El Financiero, Result: 31
Date: 2023-12-15, Poll: Simo, Result: 23
Date: 

In [14]:
# Create a set of tuples associating dates with corresponding polls
date_poll_set_maynez = set(zip(date_strings, polls, targets_maynez))
for i, n, j in date_poll_set_maynez:
  print(f"Date: {i}, Poll: {n}, Result: {j}")
print(len(date_poll_set_maynez))

Date: 2023-10-15, Poll: Parametría, Result: 9
Date: 2023-12-15, Poll: Simo, Result: 9
Date: 2023-10-15, Poll: Enkoll, Result: 15
Date: 2024-01-15, Poll: Simo, Result: 4
Date: 2023-12-15, Poll: Berumen, Result: 3
Date: 2023-11-15, Poll: El Financiero, Result: 7
Date: 2023-11-15, Poll: Mendoza Blanco, Result: 8
Date: 2023-12-15, Poll: Parametría, Result: 5
Date: 2024-01-15, Poll: Demotecnia, Result: 2
Date: 2023-12-15, Poll: Citibanamex, Result: 11
Date: 2023-12-15, Poll: GEA-ISA, Result: 10
Date: 2023-11-15, Poll: Buendía y Márquez, Result: 8
Date: 2023-09-15, Poll: Enkoll, Result: 18
Date: 2024-01-15, Poll: Mendoza Blanco, Result: 5
Date: 2023-11-15, Poll: Covarrubias, Result: 7
Date: 2024-01-15, Poll: El Financiero, Result: 10
Date: 2023-10-15, Poll: Mitofsky, Result: 10
Date: 2023-09-15, Poll: GEA-ISA, Result: 7
Date: 2024-01-15, Poll: Mitofsky, Result: 5
Date: 2023-12-15, Poll: Demotecnia, Result: 6
Date: 2023-11-15, Poll: Reforma, Result: 14
Date: 2023-12-15, Poll: Mitofsky, Result

In [16]:
# File Path
file_path = "../New_DB.xlsx"
# Read all sheets into a dictionary of DataFrames
all_sheets = pd.read_excel(file_path, skiprows=1, sheet_name=None)

# Access each DataFrame by sheet name
galvez_df = all_sheets["Galvez"]
claudia_df = all_sheets["Claudia"]
maynez_df = all_sheets["Maynez"]

# Convert 'Date' column to datetime if it's not already
galvez_df['Date'] = pd.to_datetime(galvez_df['Date'])
claudia_df['Date'] = pd.to_datetime(claudia_df['Date'])
maynez_df['Date'] = pd.to_datetime(maynez_df['Date'])

In [19]:
# How mauch data should we have?
data = len(days) * len(polls)
count = 0
for i in days:
  for x, y, z in date_poll_set_claudia:
    count += 1

print(f"We should have {data} 'records' And we have {count}")

We should have 350 'records' And we have 350


In [21]:
# CLAUDIA

# Specify the directory where you want to save the CSV files
output_directory = "../claudia"
result_dataframes = {}
for i in days:
  for x, y, z in date_poll_set_claudia:
    # Filter rows based on the established date and count the specified number of rows before that
    filtered_df = claudia_df.loc[claudia_df['Date'] < x].iloc[-i:]
    # Sum the data in the selected rows
    sum_result = filtered_df[columns].sum()
    # Turn result into a DF and Transpose
    pd_result = sum_result.to_frame().T
    #Adding metadata columns to the new sum dataframe
    pd_result['Candidate'] = 'Claudia'
    pd_result['Window'] = i
    pd_result['Ref. Date'] = x
    pd_result['Institute'] = y
    pd_result['Target'] = z
    # round the result and output
    pd_result = pd_result.round()

    # Store the dataframe in the dictionary with a meaningful key
    key = f"1_{i}_{x}_{y}"
    result_dataframes[key] = pd_result

    # Save the dataframe to a CSV file in the specified directory
    csv_filepath = os.path.join(output_directory, f"{key}.csv")
    pd_result.to_csv(csv_filepath, index=False)


In [22]:
# Specify the directory where you want to save the CSV files
output_directory = "../galvez"
result_dataframes = {}
for i in days:
  for x, y, z in date_poll_set_galvez:
    # Filter rows based on the established date and count the specified number of rows before that
    filtered_df = galvez_df.loc[galvez_df['Date'] < x].iloc[-i:]
    # Sum the data in the selected rows
    sum_result = filtered_df[columns].sum()
    # Turn result into a DF and Transpose
    pd_result = sum_result.to_frame().T
    #Adding metadata columns to the new sum dataframe
    pd_result['Candidate'] = 'Galvez'
    pd_result['Window'] = i
    pd_result['Ref. Date'] = x
    pd_result['Institute'] = y
    pd_result['Target'] = z
    # round the result and output
    pd_result = pd_result.round()

    # Store the dataframe in the dictionary with a meaningful key
    key = f"2_{i}_{x}_{y}"
    result_dataframes[key] = pd_result

    # Save the dataframe to a CSV file in the specified directory
    csv_filepath = os.path.join(output_directory, f"{key}.csv")
    pd_result.to_csv(csv_filepath, index=False)


We have now created single column files that correspond to each poll by a date, by candidate. We now need to join corresponding poll results categorized by window time.

In [25]:
# Set the directory where your CSV files are located
directory = '../claudia'
for i in days:

    # Specify the pattern for file names to include
    file_name_pattern = f'1_{i}_'

    # Get a list of CSV files in the directory that match the pattern
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv') and file.startswith(file_name_pattern)]

    # Ensure there are matching CSV files in the directory
    if not csv_files:
        print(f"No CSV files matching the pattern {file_name_pattern} found in the specified directory.")
    else:
        # Read the first CSV file to get the header
        first_file_path = os.path.join(directory, csv_files[0])
        df_combined = pd.read_csv(first_file_path)

        # Loop through the remaining CSV files and concatenate them
        for csv_file in csv_files[1:]:
            file_path = os.path.join(directory, csv_file)
            df = pd.read_csv(file_path)
            df_combined = pd.concat([df_combined, df], ignore_index=True)

        # Write the combined DataFrame to a new CSV file
        combined_output_path = f'../claudia/1_{i}.csv'
        df_combined.to_csv(combined_output_path, index=False)

        print(f"CSV files matching the pattern {file_name_pattern} successfully combined. Output saved to: {combined_output_path}")

CSV files matching the pattern 1_1_ successfully combined. Output saved to: ../claudia/1_1.csv
CSV files matching the pattern 1_2_ successfully combined. Output saved to: ../claudia/1_2.csv
CSV files matching the pattern 1_3_ successfully combined. Output saved to: ../claudia/1_3.csv
CSV files matching the pattern 1_4_ successfully combined. Output saved to: ../claudia/1_4.csv
CSV files matching the pattern 1_5_ successfully combined. Output saved to: ../claudia/1_5.csv
CSV files matching the pattern 1_6_ successfully combined. Output saved to: ../claudia/1_6.csv
CSV files matching the pattern 1_7_ successfully combined. Output saved to: ../claudia/1_7.csv
CSV files matching the pattern 1_14_ successfully combined. Output saved to: ../claudia/1_14.csv
CSV files matching the pattern 1_21_ successfully combined. Output saved to: ../claudia/1_21.csv
CSV files matching the pattern 1_28_ successfully combined. Output saved to: ../claudia/1_28.csv


In [26]:
# Set the directory where your CSV files are located
directory = '../galvez/'
for i in days:

    # Specify the pattern for file names to include
    file_name_pattern = f'2_{i}_'

    # Get a list of CSV files in the directory that match the pattern
    csv_files = [file for file in os.listdir(directory) if file.endswith('.csv') and file.startswith(file_name_pattern)]

    # Ensure there are matching CSV files in the directory
    if not csv_files:
        print(f"No CSV files matching the pattern {file_name_pattern} found in the specified directory.")
    else:
        # Read the first CSV file to get the header
        first_file_path = os.path.join(directory, csv_files[0])
        df_combined = pd.read_csv(first_file_path)

        # Loop through the remaining CSV files and concatenate them
        for csv_file in csv_files[1:]:
            file_path = os.path.join(directory, csv_file)
            df = pd.read_csv(file_path)
            df_combined = pd.concat([df_combined, df], ignore_index=True)

        # Write the combined DataFrame to a new CSV file
        combined_output_path = f'../galvez/2_{i}.csv'
        df_combined.to_csv(combined_output_path, index=False)

        print(f"CSV files matching the pattern {file_name_pattern} successfully combined. Output saved to: {combined_output_path}")


CSV files matching the pattern 2_1_ successfully combined. Output saved to: ../galvez/2_1.csv
CSV files matching the pattern 2_2_ successfully combined. Output saved to: ../galvez/2_2.csv
CSV files matching the pattern 2_3_ successfully combined. Output saved to: ../galvez/2_3.csv
CSV files matching the pattern 2_4_ successfully combined. Output saved to: ../galvez/2_4.csv
CSV files matching the pattern 2_5_ successfully combined. Output saved to: ../galvez/2_5.csv
CSV files matching the pattern 2_6_ successfully combined. Output saved to: ../galvez/2_6.csv
CSV files matching the pattern 2_7_ successfully combined. Output saved to: ../galvez/2_7.csv
CSV files matching the pattern 2_14_ successfully combined. Output saved to: ../galvez/2_14.csv
CSV files matching the pattern 2_21_ successfully combined. Output saved to: ../galvez/2_21.csv
CSV files matching the pattern 2_28_ successfully combined. Output saved to: ../galvez/2_28.csv


This process will be repeated to feed the regression models on multiple occasions (as new polls get released and as new posts get interactions on a daily basis). 

The release of this dataset belongs to the research effort performed by the author of this repository, which hopes to upbring new research based on it since similar databases have not been found.